In [ ]:
import pickle

import data_fetching
import mechanism
from data_fetching import fetch_solutions_single, fetch_solutions_batch, compute_split_solutions
from mechanism import (
    Trade,
    Solution,
    FilterRankRewardMechanism,
    VCGRewardMechanism,
    NoFilter,
    BaselineFilter,
    DirectedTokenPairs,
    TokenPairs,
    TradedTokens,
    DirectSelection,
    MonotoneSelection,
    NoReward,
    SubsetFilteringSelection,
    BatchSecondPriceReward,
    BatchOverlapSecondPriceReward,
    TokenPairImprovementReward,
    SingleSurplusSelection,
    compute_reference_solutions,
)

In [ ]:
def compute_reward_statistic(solutions_batch, mechanisms, rewards_batch):
    print(f"number of auction: {len(solutions_batch)}")

    print("\naverage number of winners per auction:")
    for k, mechanism in enumerate(mechanisms):
        print(
            f"{mechanism}: {sum(len(rewards[k]) for rewards in rewards_batch) / len(rewards_batch)}\n"
            )

    solutions_batch_dicts = [
        {solution.id: solution for solution in solutions}
        for solutions in solutions_batch
    ]
    print("\naverage total score per auction:")
    for k, mechanism in enumerate(mechanisms):
        print(
            f"{mechanism}: {sum(sum(solutions_batch_dicts[i][solution_id].score for solution_id in rewards[k]) for i, rewards in enumerate(rewards_batch)) / len(rewards_batch)}"
            )

    print("\naverage rewards per winner:")
    for k, mechanism in enumerate(mechanisms):
        print(
            f"{mechanism}: {sum(sum(reward for reward, _ in rewards[k].values()) for rewards in rewards_batch) / sum(len(rewards[0]) for rewards in rewards_batch) / 10 ** 18}")

    print("\naverage rewards/penalties:")
    for k, mechanism in enumerate(mechanisms):
        print(
            f"{mechanism}: {sum(sum(penalty for _, penalty in rewards[k].values()) for rewards in rewards_batch) / sum(len(rewards[k]) for rewards in rewards_batch) / 10 ** 18}")

    print("\norder throughput:")
    for k, mechanism in enumerate(mechanisms):
        settled = sum(len(get_settled_orders(solutions, rewards[k])) for rewards, solutions in zip(rewards_batch, solutions_batch))
        proposed = sum(len(get_orders(solutions)) for solutions in solutions_batch)
        print(
            f"{mechanism}: {settled / proposed}")

def get_settled_orders(solutions: list[Solution], rewards):
    winning_solutions = [solution for solution in solutions if solution.id in rewards.keys()]
    return get_orders(winning_solutions)

def get_orders(solutions: list[Solution]):
    return {trade.id for solution in solutions for trade in solution.trades}

def run_analysis(solutions_batch, mechanisms):
    all_rewards: list[list[dict[str, tuple[int, int]]]] = []
    for i, solutions in enumerate(solutions_batch):
        rewards = [mechanism.winners_and_rewards(solutions) for mechanism in mechanisms]

        all_rewards.append(rewards)

    compute_reward_statistic(solutions_batch, mechanisms, all_rewards)

    return all_rewards

In [ ]:
solutions_batch = [
    [  # batch vs single order solutions
        Solution(
            "batch winner",
            solver="solver 1",
            score=200,
            trades=[Trade("1", "A", "B", 100), Trade("2", "C", "D", 100)],
        ),
        Solution(
            "best on first trade",
            solver="solver 2",
            score=150,
            trades=[Trade("1", "A", "B", 150)],
        ),
        Solution(
            "best on second trade",
            solver="solver 3",
            score=150,
            trades=[Trade("2", "C", "D", 150)],
        ),
    ],
    [  # solutions without overlap
        Solution(
            "best on first trade",
            solver="solver 1",
            score=150,
            trades=[Trade("1", "A", "B", 150)],
        ),
        Solution(
            "best on second trade",
            solver="solver 2",
            score=140,
            trades=[Trade("2", "C", "D", 140)],
        ),
        Solution(
            "bad batch",
            solver="solver 3",
            score=100,
            trades=[Trade("1", "A", "B", 50), Trade("2", "C", "D", 50)],
        ),
    ],
    [  # batch in between solutions without overlap
        Solution(
            "best on first trade",
            solver="solver 1",
            score=150,
            trades=[Trade("1", "A", "B", 150)],
        ),
        Solution(
            "batch with overlap",
            solver="solver 3",
            score=100,
            trades=[Trade("1", "A", "B", 50), Trade("2", "C", "D", 50)],
        ),
        Solution(
            "best on second trade",
            solver="solver 2",
            score=90,
            trades=[Trade("2", "C", "D", 90)],
        ),
    ],
    [  # reference is not from winner
        Solution(
            "batch with overlap",
            solver="solver 1",
            score=200,
            trades=[Trade("1", "A", "B", 150), Trade("2", "C", "D", 50)],
        ),
        Solution(
            "best on first trade",
            solver="solver 1",
            score=100,
            trades=[Trade("1", "A", "B", 100)],
        ),
        Solution(
            "best on second trade",
            solver="solver 2",
            score=90,
            trades=[Trade("2", "C", "D", 90)],
        ),
    ],
    [  # token overlap but not on the same token pair
        Solution(
            "batch with overlap",
            solver="solver 1",
            score=100,
            trades=[Trade("1", "A", "B", 100)],
        ),
        Solution(
            "best on first trade",
            solver="solver 2",
            score=90,
            trades=[Trade("1", "A", "C", 90)],
        ),
    ],
    [
        Solution(
            id="batch winner",
            solver="solver 1",
            score=150,
            trades=[Trade("1", "A", "B", 100), Trade("2", "A", "C", 50)],
        ),
        Solution(
            id="unfair batch",
            solver="solver 2",
            score=110,
            trades=[Trade("1", "A", "B", 50), Trade("2", "A", "C", 60)],
        ),
        Solution(
            id="overlapping batch",
            solver="solver 3",
            score=100,
            trades=[Trade("3", "B", "A", 50), Trade("2", "A", "C", 50)],
        ),
        Solution(
            id="non-overlapping batch",
            solver="solver 4",
            score=100,
            trades=[Trade("3", "B", "A", 40), Trade("4", "D", "E", 60)],
        ),
        Solution(
            id="non-overlapping batch unfair",
            solver="solver 5",
            score=120,
            trades=[Trade("3", "B", "A", 20), Trade("4", "D", "E", 100)],
        ),
        Solution(
            id="reference A->B",
            solver="solver 1",
            score=80,
            trades=[Trade("1", "A", "B", 80)],
        ),
        Solution(
            id="reference A->C",
            solver="solver 2",
            score=40,
            trades=[Trade("2", "A", "C", 40)],
        ),
        Solution(
            id="runner up A->B",
            solver="solver 2",
            score=40,
            trades=[Trade("1", "A", "B", 40)],
        ),
        Solution(
            id="runner up A->C",
            solver="solver 1",
            score=40,
            trades=[Trade("2", "A", "C", 40)],
        ),
        Solution(
            id="reference B->A",
            solver="solver 7",
            score=30,
            trades=[Trade("3", "B", "A", 30)],
        ),
        Solution(
            id="reference F->G",
            solver="solver 8",
            score=50,
            trades=[Trade("5", "F", "G", 50)],
        ),
        Solution(
            id="runner up F->G",
            solver="solver 1",
            score=40,
            trades=[Trade("5", "F", "G", 40)],
        ),
        Solution(
            id="reference H->I",
            solver="solver 8",
            score=50,
            trades=[Trade("6", "H", "I", 50)],
        ),
    ],
]
solution_batches_split = [compute_split_solutions(solutions) for solutions in solutions_batch]

In [ ]:
# fetch auctions from file or api
auction_start = 10322553 - 50000
auction_end = 10322553
try:
    with open(f"batches_{auction_start}_{auction_end}.pickle", 'rb') as handle:
        solutions_batch = pickle.load(handle)
except FileNotFoundError:
    solutions_batch = fetch_solutions_batch(auction_start, auction_end)
    with open(f"batches_{auction_start}_{auction_end}.pickle", "wb") as handle:
        pickle.dump(solutions_batch, handle, protocol=-1)
solutions_batch_split = [compute_split_solutions(solutions) for solutions in solutions_batch]

In [ ]:
filtering_function = TradedTokens()
mechanisms = [
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(SingleSurplusSelection()),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        BaselineFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        MonotoneSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        BaselineFilter(),
        MonotoneSelection(
            SubsetFilteringSelection(
                filtering_function=filtering_function, cumulative_filtering=False
            )
        ),
        NoReward(),
    )]

In [ ]:
all_rewards = run_analysis(solutions_batch, mechanisms)

In [ ]:
all_rewards = run_analysis(solutions_batch_split, mechanisms)

In [ ]:
solutions_batch = [
    [
        Solution(
            id="batch winner",
            solver="solver 1",
            score=250,
            trades=[Trade("1", "A", "B", 150), Trade("2", "C", "D", 100)],
        ),
        Solution(
            id="overlapping batch",
            solver="solver 2",
            score=240,
            trades=[Trade("2", "C", "D", 140), Trade("3", "E", "F", 100)],
        ),
    ]
]
solutions_batch_split = [compute_split_solutions(solutions) for solutions in solutions_batch]

mechanisms = [
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(SingleSurplusSelection()),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=False
            )
        ),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        MonotoneSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=False
            )
        ),
        NoReward(),
    ),
]

run_analysis(solutions_batch_split, mechanisms)

In [ ]:
mechanism.SolverFilterBatches("solver 1").filter(solutions_batch_split[0])

In [ ]:
sum(rewards[1] != rewards[0] for rewards in all_rewards) / len(all_rewards)

In [ ]:
list(filter((lambda x: (x[1][1] != x[1][3] and x[1][2] != x[1][4])), enumerate(all_rewards)))

In [ ]:
len(list(filter((lambda x: x[1][3] != x[1][1]), enumerate(all_rewards)))) / len(all_rewards)

In [ ]:
len(list(filter((lambda x: x[1][4] != x[1][2]), enumerate(all_rewards)))) / len(all_rewards)

In [ ]:
len(solutions_batch)

In [ ]:
mechanisms[2].winners_and_rewards(solutions_batch[34803])

In [ ]:
mechanisms[2].solution_filter.filter(solutions_batch[34803])

In [ ]:
compute_reference_solutions(solutions_batch[34803])

In [ ]:
# comparison of overlap filtering
mechanisms = [
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(SingleSurplusSelection()),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TradedTokens(), cumulative_filtering=True
            )
        ),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=TokenPairs(), cumulative_filtering=True
            )
        ),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        DirectSelection(
            SubsetFilteringSelection(
                filtering_function=DirectedTokenPairs(), cumulative_filtering=True
            )
        ),
        NoReward(),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        MonotoneSelection(
            SubsetFilteringSelection(
                filtering_function=DirectedTokenPairs(), cumulative_filtering=True
            )
        ),
        NoReward(),
    ),
]
all_rewards: list[list[dict[str, tuple[int, int]]]] = []
for solutions in solutions_batch:
    rewards = [mechanism.winners_and_rewards(solutions) for mechanism in mechanisms]

    all_rewards.append(rewards)

compute_reward_statistic(solutions_batch, mechanisms, all_rewards)

In [ ]:
solutions

run_analysis(solutions_batch, mechanisms)
